In [1]:
"""This may be useless since ownership data is pretty delayed. Might be a good candidate for processing stream 
data (stock orders) to look for big insitutaional trades (Basically huge spikes in volume?)
Better idea is tracking insiders via FinViz Scraping.
"""
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import datetime


In [80]:
"""Get data from FinViz and parse to list of lists"""
#this url is just for the buys, if you replace tc=1 with tc=2 it will be the sells and tc=7 for all
finviz_url = 'https://finviz.com/insidertrading.ashx?tc=1'

req = Request(url=finviz_url,headers={'user-agent': 'my-app/0.0.1'}) 
response = urlopen(req)    

# Read the contents of the file into 'html'
html = BeautifulSoup(response)


# Find gold in the Soup
buys1 = html.body.find_all('tr', attrs={'class':'insider-buy-row-1'})
buys2 = html.body.find_all('tr', attrs={'class':'insider-buy-row-2'})
buys1.extend(buys2)

buy_data = []
for buy in buys1:
    inner_list = []
    if str(buy.find_all('td', attrs={'style':'white-space:nowrap'})[3]).split('>')[1].split('</td')[0].lower() == 'buy':
    
        ticker = str(buy.find('a')).split('>')[1].split('</a')[0]
        inner_list += [ticker]

        date = str(buy.find_all('td', attrs={'style':'white-space:nowrap'})[2]).split('>')[1].split('</td')[0]
        inner_list += [date]

        filing = str(buy.find_all('td', attrs={'style':'white-space:nowrap'})[4]).split('>')[2].split('</a')[0]
        inner_list += [filing]
        
        cost = str(buy.find_all('td', attrs={'align':'right'})[0]).split('>')[1].split('</td')[0]
        inner_list += [cost]
        
        shares = str(buy.find_all('td', attrs={'align':'right'})[1]).split('>')[1].split('</td')[0]
        inner_list += [shares]
        
        value = str(buy.find_all('td', attrs={'align':'right'})[2]).split('>')[1].split('</td')[0]
        inner_list += [value]
        
        shares_total = str(buy.find_all('td', attrs={'align':'right'})[3]).split('>')[1].split('</td')[0]
        inner_list += [shares_total]
        
        buy_data += [inner_list] 
    else:
        pass
    

In [98]:
"""Create DataFrame and fix up some columns (the date column is a little funky since it comes as %b %d)"""
cols = ['ticker','date', 'filing', 'cost', 'shares', 'value', 'shares_total']
df_buys = pd.DataFrame(buy_data, columns = cols) 

df_buys.loc[:,"date"] = [datetime.datetime.strptime(x+' 2020','%b %d %Y') for x in df_buys["date"]]
df_buys.loc[:,"filing"] = [datetime.datetime.strptime(x+' 2020','%b %d %I:%M %p %Y') for x in df_buys["filing"]]
df_buys.loc[:,"cost"] = df_buys["cost"].astype(float)
df_buys.loc[:,"shares"] = [pd.to_numeric(x.replace(',', '')) for x in df_buys["shares"]]
df_buys.loc[:,"value"] = [pd.to_numeric(x.replace(',', '')) for x in df_buys["value"]]
df_buys.loc[:,"shares_total"] = [pd.to_numeric(x.replace(',', '')) for x in df_buys["shares_total"]]
df_buys.head()

,ticker,date,filing,cost,shares,value,shares_total
0,RVP,2020-10-09,2020-10-09 17:03:00,8.24,400,3296,14064389
1,GBDC,2020-10-07,2020-10-09 16:55:00,13.54,2000,27076,1844890
2,GBDC,2020-10-08,2020-10-09 16:55:00,13.78,2000,27560,1846890
3,GBDC,2020-10-07,2020-10-09 16:54:00,13.54,2000,27076,1844890
4,GBDC,2020-10-08,2020-10-09 16:54:00,13.78,2000,27560,1846890


In [118]:
# Work on fixing the date-stamps
now = datetime.datetime(2021,1,8)
if now - datetime.datetime(2020, 12, 1) < datetime.datetime(2021, 12, 1) - now:
    print("using 2020")
else:
    print("using 2021")

using 2020
